In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    image.flags.writeable = False                  
    results = model.process(image)                 
    image.flags.writeable = True                    
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [5]:
def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             )  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [12]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        print(results)

        draw_styled_landmarks(image, results)
        
        cv2.imshow('OpenCV Feed', cv2.flip(image, 1))

        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [6]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.array([-2]*63)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.array([-2]*63)
    return np.concatenate([lh, rh])

In [13]:
extract_keypoints(results)

array([ 7.62771308e-01,  5.93124211e-01,  2.02406071e-07,  7.23323107e-01,
        5.74292719e-01, -1.02159483e-02,  6.87009692e-01,  5.41721761e-01,
       -1.57207157e-02,  6.60077214e-01,  5.19528210e-01, -2.11223196e-02,
        6.36046290e-01,  5.09926558e-01, -2.66955215e-02,  7.08672166e-01,
        4.65741813e-01, -3.47708818e-03,  6.91331387e-01,  4.19770509e-01,
       -1.13185570e-02,  6.82037950e-01,  3.93116534e-01, -1.96667146e-02,
        6.75021112e-01,  3.69935691e-01, -2.63079386e-02,  7.31882215e-01,
        4.54979509e-01, -6.98322244e-03,  7.23604858e-01,  3.99774104e-01,
       -1.19554419e-02,  7.19277024e-01,  3.67581606e-01, -1.84672475e-02,
        7.16064811e-01,  3.40965509e-01, -2.39188988e-02,  7.55421937e-01,
        4.57363993e-01, -1.27501460e-02,  7.52267957e-01,  4.04943436e-01,
       -1.90728717e-02,  7.50622094e-01,  3.76007795e-01, -2.34802943e-02,
        7.48981297e-01,  3.50812674e-01, -2.72642728e-02,  7.78147578e-01,
        4.69861746e-01, -

In [7]:
DATA_PATH = os.path.join('Data')
actions = ['Hello', 'Thanks']
no_sequences = 30
no_frames = 30
start_folder = 30

In [19]:
for action in actions: 
    for sequence in range(1, no_sequences + 1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [20]:
def addMoreVideos(no_sequences):
    for action in actions:
        dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
        print(action, dirmax, sep='\n')
        for sequence in range(1, no_sequences + 1):
            try:
                os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
                print(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
            except:
                pass

In [84]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(
    min_detection_confidence = 0.5,
    min_tracking_confidence = 0.5 ) as holistic:

    # for each action
    for action in actions:
        # for each video
        for sequence in range(1, no_sequences+1):
            # for each frame:
            for frame_num in range(no_frames):
                ret, frame = cap.read()

                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)

                image = cv2.flip(image, 1)
                if frame_num == 0:
                    cv2.putText(image, f'STARTING COLLECTION FOR {action}, video {sequence}', (120, 200),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)

                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                # print(npy_path)
                np.save(npy_path, keypoints)

                if cv2.waitKey(5) & 0xFF == 27:
                    break
    cap.release()
    cv2.destroyAllWindows()


KeyboardInterrupt: 

In [86]:
cap.release()
cv2.destroyAllWindows()

In [14]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [15]:
label_map = {label: num for num, label in enumerate(actions)}
label_map

{'Hello': 0, 'Thanks': 1}

In [17]:
sequences, labels = [], []
for action in actions:
    for sequence in range(1, no_sequences+1):
        window = []
        for frame_num in range(no_frames):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [18]:
np.array(sequences).shape

(60, 30, 126)

In [19]:
np.array(labels).shape

(60,)

In [20]:
X = np.array(sequences)
X.shape

(60, 30, 126)

In [21]:
y = to_categorical(labels).astype(int)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [26]:
y_test.shape

(6, 2)

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [28]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [37]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(np.array(actions).shape[0], activation='softmax'))

In [38]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [39]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
2/2 [==============================] - 5s 436ms/step - loss: 6.4309 - categorical_accuracy: 0.5926
Epoch 2/2000
2/2 [==============================] - 0s 177ms/step - loss: 0.6654 - categorical_accuracy: 0.8148
Epoch 3/2000
2/2 [==============================] - 0s 105ms/step - loss: 0.3501 - categorical_accuracy: 1.0000
Epoch 4/2000
2/2 [==============================] - 0s 160ms/step - loss: 0.2941 - categorical_accuracy: 1.0000
Epoch 5/2000
2/2 [==============================] - 0s 101ms/step - loss: 0.2556 - categorical_accuracy: 1.0000
Epoch 6/2000
2/2 [==============================] - 0s 175ms/step - loss: 0.1880 - categorical_accuracy: 1.0000
Epoch 7/2000
2/2 [==============================] - 0s 102ms/step - loss: 0.0577 - categorical_accuracy: 1.0000
Epoch 8/2000
2/2 [==============================] - 0s 102ms/step - loss: 0.0037 - categorical_accuracy: 1.0000
Epoch 9/2000
2/2 [==============================] - 0s 107ms/step - loss: 1.1931e-04 - categorical_accur

In [40]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_6 (LSTM)               (None, 30, 64)            48896     
                                                                 
 lstm_7 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_5 (Dense)             (None, 64)                4160      
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 2)                 66        
                                                                 
Total params: 203,426
Trainable params: 203,426
Non-tr

In [41]:
res = model.predict(X_test)

In [42]:
actions[np.argmax(res[4])]

'Hello'

In [43]:
actions[np.argmax(y_test[4])]

'Hello'

In [44]:
model.save('action.h5')

In [45]:
del model

In [50]:
import tensorflow as tf
model = tf.keras.models.load_model('action.h5')

In [62]:
from sklearn.metrics import confusion_matrix, accuracy_score
ytrue = np.argmax(y_test, axis = 1).tolist()

yhat = model.predict(X_test)
yhat = np.argmax(yhat, axis = 1).tolist()

confusion_matrix(ytrue, yhat)

array([[3, 0],
       [1, 2]], dtype=int64)

In [63]:
accuracy_score(ytrue, yhat)

0.8333333333333334

In [79]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [80]:
sequences = [] # once we got 30 frames we will make prediction
sentence = [] # All Detections

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        # print(results)

        draw_styled_landmarks(image, results)

        # Collecting 30 frames
        keypoints = extract_keypoints(results)
        sequences.append(keypoints)
        sequences = sequences[-30:] # Extract last 30 frames to make prediction

        if len(sequences) == 30:
            res = model.predict(np.expand_dims(sequences, axis = 0))[0]
            print(actions[np.argmax(res)], np.max(res))
        
            sentence.append(actions[np.argmax(res)])

            print(sentence)
            if len(sentence) > 5: 
                sentence = sentence[-5:]
            
            image = cv2.flip(image, 1)
            image = prob_viz(res, actions, image, colors)

            

            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow('OpenCV Feed', image)

        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()

Hello 1.0
['Hello']
Hello 1.0
['Hello', 'Hello']
Hello 1.0
['Hello', 'Hello', 'Hello']
Hello 1.0
['Hello', 'Hello', 'Hello', 'Hello']
Hello 1.0
['Hello', 'Hello', 'Hello', 'Hello', 'Hello']
Hello 1.0
['Hello', 'Hello', 'Hello', 'Hello', 'Hello', 'Hello']
Hello 1.0
['Hello', 'Hello', 'Hello', 'Hello', 'Hello', 'Hello']
Hello 1.0
['Hello', 'Hello', 'Hello', 'Hello', 'Hello', 'Hello']
Hello 0.703528
['Hello', 'Hello', 'Hello', 'Hello', 'Hello', 'Hello']
Hello 1.0
['Hello', 'Hello', 'Hello', 'Hello', 'Hello', 'Hello']
Hello 1.0
['Hello', 'Hello', 'Hello', 'Hello', 'Hello', 'Hello']
Hello 1.0
['Hello', 'Hello', 'Hello', 'Hello', 'Hello', 'Hello']
Hello 1.0
['Hello', 'Hello', 'Hello', 'Hello', 'Hello', 'Hello']
Hello 1.0
['Hello', 'Hello', 'Hello', 'Hello', 'Hello', 'Hello']
Hello 1.0
['Hello', 'Hello', 'Hello', 'Hello', 'Hello', 'Hello']
Hello 1.0
['Hello', 'Hello', 'Hello', 'Hello', 'Hello', 'Hello']
Hello 1.0
['Hello', 'Hello', 'Hello', 'Hello', 'Hello', 'Hello']
Hello 1.0
['Hello', 'Hell

In [73]:
cap.release()
cv2.destroyAllWindows()

In [83]:
np.load(os.path.join(DATA_PATH, 'Hello', '1', '0.npy')).shape

(126,)